<a href="https://colab.research.google.com/github/Yashcode007/pytorch/blob/main/bert_finetuning_model_sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Both type of model generative and representation models can be used for text classification
In Representation Model , we can define it by classes 0,,1,2 and in the Generative Model , we can output that the following is of the class Returns , New etc .

**1) Movie Review Sentiment Analysis**

Within the Representation Models , there are 2 ways we can do the classification :-
1) Task Specific
2) Embedding Method

Method to solve for the Task Specific Method:-
Base (BERT) -> Fine-tuning (classification) -> Task-specific classification

For the Embedding based method:-
Base(BERT) -> Convert them into embeddings, then we will apply the logistic regression or any ml algorithm based on the embedding will judge whether it is a hate speech or not

In [3]:
! pip install datasets transformers

**Data**

In [4]:
from datasets import load_dataset

data = load_dataset("rotten_tomatoes")
data

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [5]:
data["train"][0,-1]

{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'things really get weird , though not particularly scary : the movie is all portent and no content .'],
 'label': [1, 0]}

Model selection is not an easy task:-
- 60000 + models for text classification
- 8000 models for generating embeddings

Good Starting Points:-
1) BERT base model(uncased)
2) RoBERTa base model
3) DistilBERT base model(uncased)
4) DeBERTa base model
5) bert-tiny
6) ALBERT base v2

Text Classification with Representation Models

Using a Task-Specific Model

In [6]:
from transformers import pipeline

model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

pipe = pipeline(
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    devices="cuda:0"
)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


tqdm is a Python library used to create progress bars for loops and long-running operations.
It’s super helpful when you're processing large datasets, training models, or running time-consuming loops.

In [7]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

y_pred=[]
for output in tqdm(pipe(KeyDataset(data["test"], "text")), total=len(data["test"])):
  negative_score = output[0]["score"]
  positive_score = output[2]["score"]
  assignment = np.argmax([negative_score, positive_score])
  y_pred.append(assignment)


  0%|          | 0/1066 [00:00<?, ?it/s]


TypeError: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'devices'

In [1]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
  performance = classification_report(
      y_true, y_pred,
      target_names=["Negative Review","Positive Review"]
  )
  print(performance)